In [10]:
import os
import torch
import torchvision
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from datetime import datetime
import numpy as np
import os
import random
import argparse
from PIL import Image
import csv

In [21]:
# dataset definition
class myDataset(Dataset):
    # load the dataset
    def __init__(self,root,transforms=None):
        self.root = root
        self.transforms = transforms
        self.data = open("/home/semone/notebook/soft/annotations.csv")
        self.imgs = list(sorted(os.listdir(os.path.join(root, "imgs"))))


    # get a row at an index
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "imgs", self.imgs[idx])
        #mask_path = os.path.join(self.root, "annos", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        
        data = csv.reader(self.data)
        
        boxes = []
        row = data.__next__()
        
        for x in range(4):
            row = data.__next__()
            x1 = int(row[1])
            y1 = int(row[2])
            x2 = int(row[3])
            y2 = int(row[4])
            boxes.append([x1, y1, x2, y2])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones(1, dtype=torch.int64)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [12]:
import detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.Normalize())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [23]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn()

In [24]:
import detection.utils as utils
dataset = myDataset('/home/semone/db_lisa_tiny', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, collate_fn=utils.collate_fn)

In [35]:
test_img, test_target = dataset.__getitem__(0)

In [36]:
print(test_target)

{'boxes': tensor([[546., 300., 571., 325.],
        [493., 295., 515., 317.],
        [132., 102., 186., 151.],
        [192., 302., 211., 321.]]), 'labels': tensor([1]), 'image_id': tensor([0]), 'area': tensor([ 625.,  484., 2646.,  361.])}


In [37]:
model([test_img], [test_target])

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [34]:
targets

[{'boxes': tensor([[190.,  40., 211.,  63.],
          [  4., 246.,  43., 283.],
          [389., 286., 418., 314.],
          [307., 243., 315., 251.]]),
  'labels': tensor([1]),
  'image_id': tensor([0]),
  'area': tensor([ 483., 1443.,  812.,   64.])},
 {'boxes': tensor([[190., 137., 228., 174.],
          [650.,  26., 674.,  50.],
          [297., 289., 316., 309.],
          [176.,  17., 215.,  53.]]),
  'labels': tensor([1]),
  'image_id': tensor([1]),
  'area': tensor([1406.,  576.,  380., 1404.])}]

In [38]:
test_img

tensor([[[0.2706, 0.2980, 0.2863,  ..., 0.2431, 0.2235, 0.2235],
         [0.2863, 0.3059, 0.2863,  ..., 0.2510, 0.2353, 0.2353],
         [0.3020, 0.3176, 0.2863,  ..., 0.2549, 0.2549, 0.2627],
         ...,
         [0.3804, 0.3804, 0.3843,  ..., 0.0235, 0.0157, 0.0157],
         [0.3804, 0.3804, 0.3843,  ..., 0.0000, 0.0000, 0.0000],
         [0.3804, 0.3804, 0.3843,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2235, 0.2510, 0.2392,  ..., 0.2471, 0.2353, 0.2353],
         [0.2392, 0.2588, 0.2392,  ..., 0.2549, 0.2471, 0.2471],
         [0.2627, 0.2784, 0.2471,  ..., 0.2588, 0.2667, 0.2745],
         ...,
         [0.3961, 0.3961, 0.4000,  ..., 0.0235, 0.0157, 0.0157],
         [0.3961, 0.3961, 0.4000,  ..., 0.0000, 0.0000, 0.0000],
         [0.3961, 0.3961, 0.4000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.2235, 0.2510, 0.2392,  ..., 0.2157, 0.2039, 0.2039],
         [0.2392, 0.2588, 0.2392,  ..., 0.2235, 0.2157, 0.2157],
         [0.2588, 0.2745, 0.2431,  ..., 0.2275, 0.2353, 0.

In [157]:
x = "190"
y = int(x)

In [158]:
y

190

In [174]:
import sys
!{sys.executable} -m pip install sgmllib3k

  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=e987e377a89b1d94c24e5b121c344821ab68f92e366a0340c09082bf2b320b3d
  Stored in directory: /home/semone/.cache/pip/wheels/83/63/2f/117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built sgmllib3k


In [183]:
import detection.transforms as T

ModuleNotFoundError: No module named 'detection.transforms'

In [33]:
dataset.__getitem__(3)

(tensor([[[0.1608, 0.1608, 0.1608,  ..., 0.1961, 0.1882, 0.1882],
          [0.1608, 0.1608, 0.1608,  ..., 0.1961, 0.1882, 0.1882],
          [0.1608, 0.1608, 0.1608,  ..., 0.2000, 0.1961, 0.1961],
          ...,
          [0.0667, 0.0667, 0.0667,  ..., 0.0196, 0.0196, 0.0196],
          [0.0667, 0.0667, 0.0667,  ..., 0.0196, 0.0196, 0.0196],
          [0.0667, 0.0667, 0.0667,  ..., 0.0196, 0.0196, 0.0196]],
 
         [[0.4510, 0.4510, 0.4510,  ..., 0.5059, 0.4980, 0.4980],
          [0.4510, 0.4510, 0.4510,  ..., 0.5059, 0.4980, 0.4980],
          [0.4510, 0.4510, 0.4510,  ..., 0.5098, 0.5059, 0.5059],
          ...,
          [0.1412, 0.1412, 0.1412,  ..., 0.0588, 0.0588, 0.0588],
          [0.1412, 0.1412, 0.1412,  ..., 0.0588, 0.0588, 0.0588],
          [0.1412, 0.1412, 0.1412,  ..., 0.0588, 0.0588, 0.0588]],
 
         [[0.7020, 0.7020, 0.7020,  ..., 0.7686, 0.7608, 0.7608],
          [0.7020, 0.7020, 0.7020,  ..., 0.7686, 0.7608, 0.7608],
          [0.7020, 0.7020, 0.7020,  ...,